## Amazon Bedrock Guardrails - Examples using the Python SDK

Amazon Bedrock Guardrails provides safeguards that you can configure for your generative AI applications based on your use cases and responsible AI policies. You can create multiple guardrails tailored to different use cases and apply them across multiple foundation models (FMs), providing a consistent user experience and standardizing safety and privacy controls across generative AI applications. You can use guardrails for both model prompts and responses with natural language.

**Key Benefits:**
- **Consistent Safety**: Apply the same safety standards across multiple foundation models
- **Customizable Protection**: Tailor guardrails to your specific use case and industry requirements
- **Real-time Filtering**: Filter both input prompts and model responses in real-time
- **Compliance Support**: Help meet regulatory and organizational compliance requirements

**Core Components:**
1. **Content Filters** - Block harmful content (hate, violence, sexual content, etc.)
2. **Denied Topics** - Prevent discussions on specific topics
3. **Word Filters** - Block specific words or phrases
4. **Sensitive Information Filters** - Detect and handle PII and sensitive data
5. **Contextual Grounding Checks** - Prevent hallucinations and ensure factual accuracy
6. **Automated Reasoning Checks** - Validate responses against logical rules and policies

In [ ]:
import boto3
import sys
import pprint
sys.path.append('../')
from util.tagging import standard_tags_kv_lc
print(boto3.__version__)
pp = pprint.PrettyPrinter(indent=2)

In [ ]:
client = boto3.client('bedrock')

## Creating a Guardrail

Guardrails for Amazon Bedrock have multiple components which include Content Filters, Denied Topics, Word and Phrase Filters, and Sensitive Word (PII & Regex) Filters. For a full list check out the policies to configure when you [create your guardrail](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-components.html).

### Understanding Guardrail Configuration

In this example, we're creating a guardrail specifically designed for a financial services application that needs to prevent the AI from providing fiduciary advice. This demonstrates a real-world use case where regulatory compliance is critical.

**Configuration Components Explained:**

1. **Topic Policy (Denied Topics)**:
   - Defines specific topics that should be blocked
   - Uses natural language definitions and examples
   - In our case: preventing fiduciary investment advice

2. **Content Policy (Content Filters)**:
   - Filters harmful content categories with configurable strength levels
   - Strength levels: NONE, LOW, MEDIUM, HIGH
   - Separate settings for input (user prompts) and output (model responses)

3. **Word Policy (Word Filters)**:
   - Blocks specific words/phrases (exact match)
   - Includes managed word lists (like profanity)
   - Custom words related to financial advice in our example

4. **Sensitive Information Policy**:
   - Handles PII detection and protection
   - Actions: BLOCK (reject) or ANONYMIZE (mask)
   - Supports custom regex patterns for domain-specific data

5. **Contextual Grounding Policy**:
   - Prevents hallucinations in RAG applications
   - GROUNDING: Ensures responses are based on source material
   - RELEVANCE: Ensures responses are relevant to the query

In [ ]:
create_response = client.create_guardrail(
    name='fiduciary-advice',
    description='Prevents the model from providing fiduciary advice.',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Fiduciary Advice',
                'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.',
                'examples': [
                    'What stocks should I invest in for my retirement?',
                    'Is it a good idea to put my money in a mutual fund?',
                    'How should I allocate my 401(k) investments?',
                    'What type of trust fund should I set up for my children?',
                    'Should I hire a financial advisor to manage my investments?'
                ],
                'type': 'DENY'
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'fiduciary advice'},
            {'text': 'investment recommendations'},
            {'text': 'stock picks'},
            {'text': 'financial planning guidance'},
            {'text': 'portfolio allocation advice'},
            {'text': 'retirement fund suggestions'},
            {'text': 'wealth management tips'},
            {'text': 'trust fund setup'},
            {'text': 'investment strategy'},
            {'text': 'financial advisor recommendations'}
        ],
        'managedWordListsConfig': [
            {'type': 'PROFANITY'}
        ]
    },
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {'type': 'EMAIL', 'action': 'ANONYMIZE'},
            {'type': 'PHONE', 'action': 'ANONYMIZE'},
            {'type': 'NAME', 'action': 'ANONYMIZE'},
            {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'BLOCK'},
            {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'BLOCK'},
            {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
        ],
        'regexesConfig': [
            {
                'name': 'Account Number',
                'description': 'Matches account numbers in the format XXXXXX1234',
                'pattern': r'\b\d{6}\d{4}\b',
                'action': 'ANONYMIZE'
            }
        ]
    },
    contextualGroundingPolicyConfig={
        'filtersConfig': [
            {
                'type': 'GROUNDING',
                'threshold': 0.75
            },
            {
                'type': 'RELEVANCE',
                'threshold': 0.75
            }
        ]
    },
    blockedInputMessaging="""I can provide general information about Acme Financial's products and services, but I can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
    blockedOutputsMessaging="""I can provide general information about Acme Financial's products and services, but I can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
    tags=[
        {'key': 'purpose', 'value': 'fiduciary-advice-prevention'},
        {'key': 'environment', 'value': 'production'}
    ] + standard_tags_kv_lc
)

pp.pprint(create_response)

### Analyzing the Guardrail Creation Response

The `create_guardrail` API call returns several important pieces of information:

- **guardrailId**: Unique identifier for this guardrail (e.g. `68cybpp9twmu`)
- **version**: Initially created as 'DRAFT' - allows for testing and iteration

## Guardrail Version Management

Amazon Bedrock Guardrails supports version management to enable safe iteration and deployment:

**DRAFT vs VERSIONED Guardrails:**
- **DRAFT**: Mutable version for development and testing
- **VERSIONED**: Immutable snapshots for production use
- You can have one DRAFT and multiple numbered versions (1, 2, 3, etc.)

**Best Practices:**
1. Develop and test with DRAFT versions
2. Create numbered versions for production deployment
3. Use version numbers to track changes and enable rollbacks
4. Tag versions with meaningful descriptions

In [ ]:
# Now let's create a version for our Guardrail 
version_response = client.create_guardrail_version(
    guardrailIdentifier=create_response['guardrailId'],
    description='Version of Guardrail'
)

version_response

In [ ]:
# To list the DRAFT version of all your guardrails, don't specify the guardrailIdentifier field. To list all versions of a guardrail, specify the ARN of the guardrail in the guardrailIdentifier field.
list_guardrails_response = client.list_guardrails(
    guardrailIdentifier=create_response['guardrailArn'],
    maxResults=5)

pp.pprint(list_guardrails_response)

## Understanding Guardrail Updates

Guardrail updates demonstrate the iterative nature of responsible AI development:

**Update Process:**
1. Only DRAFT versions can be updated
2. Updates modify the entire guardrail configuration
3. Changes are immediately available for testing
4. Create new versions to preserve update history

**In this example**, we're changing the HATE filter strength from HIGH to MEDIUM, demonstrating how you might fine-tune guardrail sensitivity based on testing results or changing requirements.

**Content Filter Strength Levels:**
- **NONE**: No filtering
- **LOW**: Minimal filtering, only obvious violations
- **MEDIUM**: Balanced filtering
- **HIGH**: Strict filtering, may catch borderline cases

In [ ]:
# Updating the Guardrail by providing another adjusting our Content Filter strength
response = client.update_guardrail(
    guardrailIdentifier=create_response['guardrailArn'],
    name='fiduciary-advice',
    description='Prevents the model from providing fiduciary advice.',
    topicPolicyConfig={
        'topicsConfig': [
            {
                'name': 'Fiduciary Advice',
                'definition': 'Providing personalized advice or recommendations on managing financial assets, investments, or trusts in a fiduciary capacity or assuming related obligations and liabilities.',
                'examples': [
                    'What stocks should I invest in for my retirement?',
                    'Is it a good idea to put my money in a mutual fund?',
                    'How should I allocate my 401(k) investments?',
                    'What type of trust fund should I set up for my children?',
                    'Should I hire a financial advisor to manage my investments?'
                ],
                'type': 'DENY'
            }
        ]
    },
    contentPolicyConfig={
        'filtersConfig': [
            {
                'type': 'SEXUAL',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'VIOLENCE',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'HATE',
                'inputStrength': 'MEDIUM',
                'outputStrength': 'MEDIUM'
            },
            {
                'type': 'INSULTS',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'MISCONDUCT',
                'inputStrength': 'HIGH',
                'outputStrength': 'HIGH'
            },
            {
                'type': 'PROMPT_ATTACK',
                'inputStrength': 'HIGH',
                'outputStrength': 'NONE'
            }
        ]
    },
    wordPolicyConfig={
        'wordsConfig': [
            {'text': 'fiduciary advice'},
            {'text': 'investment recommendations'},
            {'text': 'stock picks'},
            {'text': 'financial planning guidance'},
            {'text': 'portfolio allocation advice'},
            {'text': 'retirement fund suggestions'},
            {'text': 'wealth management tips'},
            {'text': 'trust fund setup'},
            {'text': 'investment strategy'},
            {'text': 'financial advisor recommendations'}
        ],
        'managedWordListsConfig': [
            {'type': 'PROFANITY'}
        ]
    },
    sensitiveInformationPolicyConfig={
        'piiEntitiesConfig': [
            {'type': 'EMAIL', 'action': 'ANONYMIZE'},
            {'type': 'PHONE', 'action': 'ANONYMIZE'},
            {'type': 'NAME', 'action': 'ANONYMIZE'},
            {'type': 'US_SOCIAL_SECURITY_NUMBER', 'action': 'BLOCK'},
            {'type': 'US_BANK_ACCOUNT_NUMBER', 'action': 'BLOCK'},
            {'type': 'CREDIT_DEBIT_CARD_NUMBER', 'action': 'BLOCK'}
        ],
        'regexesConfig': [
            {
                'name': 'Account Number',
                'description': 'Matches account numbers in the format XXXXXX1234',
                'pattern': r'\b\d{6}\d{4}\b',
                'action': 'ANONYMIZE'
            }
        ]
    },
    contextualGroundingPolicyConfig={
        'filtersConfig': [
            {
                'type': 'GROUNDING',
                'threshold': 0.75
            },
            {
                'type': 'RELEVANCE',
                'threshold': 0.75
            }
        ]
    },
    blockedInputMessaging="""I can provide general information about Acme Financial's products and services, but I can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
    blockedOutputsMessaging="""I can provide general information about Acme Financial's products and services, but I can't fully address your request here. For personalized help or detailed questions, please contact our customer service team directly. For security reasons, avoid sharing sensitive information through this channel. If you have a general product question, feel free to ask without including personal details. """,
)



In [ ]:
# Let's now get all of our updates 
get_response = client.get_guardrail(
    guardrailIdentifier=create_response['guardrailId'],
    guardrailVersion='DRAFT'
)
pp.pprint(get_response)

In [ ]:
# Create a new Version from our updates 
version_response = client.create_guardrail_version(
    guardrailIdentifier=create_response['guardrailId'],
    description='Version of Guardrail that has a MEDIUM Hate Filter'
)

In [ ]:
# Get all of our Guardrails 
list_guardrails_response = client.list_guardrails(
    guardrailIdentifier=create_response['guardrailArn'],
    maxResults=5)

## Testing our Guardrail

In [ ]:
#import the run-time client
import json
bedrock_runtime = boto3.client('bedrock-runtime')

In [ ]:
# Build our request to Bedrock, we will test our second version

payload = {
    "modelId": "us.amazon.nova-lite-v1:0",
    "contentType": "application/json",
    "accept": "application/json",
    "body": {
        "schemaVersion": "messages-v1",
        "messages": [
            {
                "role": "user",
                "content": [
                    {
                        "text": "How should I invest for my retirement? I want to be able to generate $5,000 a month"
                    }
                ]
            }
        ],
        "inferenceConfig": {
            "maxTokens": 1000
        }
    }
}

# Convert the payload to bytes
body_bytes = json.dumps(payload['body']).encode('utf-8')

# Invoke the model
response = bedrock_runtime.invoke_model(
    body = body_bytes,
    contentType = payload['contentType'],
    accept = payload['accept'],
    modelId = payload['modelId'],
    guardrailIdentifier = create_response['guardrailId'], 
    guardrailVersion ="2", 
    trace = "ENABLED"
)

# Print the response
response_body = response['body'].read().decode('utf-8')
print(json.dumps(json.loads(response_body), indent=2))

## Summary

### Core Guardrail Components

- **Content Filters**: Block harmful content (sexual, violence, hate, insults, misconduct, prompt attacks) with configurable strength levels (NONE/LOW/MEDIUM/HIGH)
- **Denied Topics**: Prevent specific business topics using natural language definitions and examples (e.g., fiduciary advice prevention)
- **Word Filters**: Exact-match blocking of custom words/phrases plus managed lists (profanity)
- **PII Protection**: Detect and handle sensitive information with BLOCK (reject) or ANONYMIZE (mask) actions
- **Contextual Grounding**: Prevent hallucinations with GROUNDING (factual accuracy) and RELEVANCE (query alignment) thresholds

### Version Management Best Practices
- **DRAFT**: Mutable versions for development and testing iterations
- **VERSIONED**: Immutable numbered snapshots for production deployment
- **Workflow**: Develop with DRAFT → Test thoroughly → Create version → Deploy to production

### Testing and Production Insights
- **Trace Analysis**: Enable `trace="ENABLED"` for detailed guardrail decision logging and debugging
- **Performance**: ~241ms processing latency, usage units consumed per policy type, 100% content coverage validation
- **Monitoring**: Track `guardrailAction` (INTERVENED/NONE), policy-specific blocking reasons, and usage metrics
- **User Experience**: Custom `blockedInputMessaging` and `blockedOutputsMessaging` for helpful user guidance
- **Real-World Application**: Multi-layered protection (topics + words + content filters) increase safety coverage